In [5]:
import sqlite3
import pandas as pd

In [8]:
conn = sqlite3.connect("../data/crypto_database.db")

In [9]:
csv_files = {
    "ETH": "..data/raw/ETH_price_data.csv",
    "XRP": "..data/raw/XRP_price_data.csv",
    "LTC": "..data/raw/LTC_price_data.csv",
    "ADA": "..data/raw/ADA_price_data.csv",
    "DOT": "..data/raw/DOT_price_data.csv",
    "LINK": "..data/raw/LINK_price_data.csv",
    "BNB": "..data/raw/BNB_price_data.csv",
    "UNI": "..data/raw/UNI_price_data.csv",
    "DOGE": "..data/raw/DOGE_price_data.csv"
}


In [11]:
tickers = list(csv_files.keys())
df_dict = dict()
for ticker in tickers:
    if ticker != 'BTC':
        query = f"""
        SELECT b.time,
               b.open as BTC_open,
               b.close as BTC_close,
               b.volumeto as BTC_volume,
               {ticker}.open as {ticker}_open,
               {ticker}.close as {ticker}_close,
               {ticker}.volumeto as {ticker}_volume
        FROM BTC as b
        INNER JOIN {ticker} as {ticker}
        ON b.time = {ticker}.time
        """
        df_dict[ticker] = pd.read_sql(query, conn)


In [12]:
for i in range(len(list(df_dict.values()))):
    list(df_dict.values())[i].to_csv(f'../data/merged_data/{list(csv_files.keys())[i]}')

In [13]:
symbols = list(csv_files.keys())

# Base query with just BTC
query = f"""
SELECT b.time,
       b.open as BTC_open,
       b.close as BTC_close,
       b.volumeto as BTC_volume
"""

# Add the columns for each table to the SELECT statement and add the necessary JOIN statements
for symbol in symbols:
    if symbol != 'BTC': # since we have already included BTC
        query += f""",
               {symbol}.open as {symbol}_open,
               {symbol}.close as {symbol}_close,
               {symbol}.volumeto as {symbol}_volume"""

query += "\nFROM BTC as b"

for symbol in symbols:
    if symbol != 'BTC': # since we have already included BTC
        query += f"""
        INNER JOIN {symbol} as {symbol}
        ON b.time = {symbol}.time"""

# Execute the query
df = pd.read_sql(query, conn)

In [14]:
df.to_csv('../data/merged_data/all_currencies.csv',index=False)